In [1]:
#!pip install -q --upgrade nltk gensim bokeh pandas
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Alexander
[nltk_data]     Solomennik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Alexander
[nltk_data]     Solomennik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# !pip install -q --upgrade gensim==3.8.2

In [3]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE

In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

## Запилим пословный машинный перевод!

In [5]:
# !wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
# %wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
# %wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
# %!pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
# downloaded.GetContentFile('cc.ru.300.vec.zip')

# downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
# downloaded.GetContentFile('cc.uk.300.vec.zip')

# %!unzip cc.ru.300.vec.zip
# %!unzip cc.uk.300.vec.zip

Напишем простенькую реализацию модели машинного перевода.

Идея основана на статье [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). У авторов в репозитории еще много интересного: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

А мы будем переводить с украинского на русский.

![](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)   
*синій кіт* vs. *синій кит*

In [6]:
from gensim.models import KeyedVectors

load_vec_file = False
if  load_vec_file:
    ru_emb = KeyedVectors.load_word2vec_format("hw3/temp_data/cc.ru.300.vec")
    uk_emb = KeyedVectors.load_word2vec_format("hw3/temp_data/cc.uk.300.vec")
if not load_vec_file:
    ru_emb = KeyedVectors.load('hw3/temp_data/ru_emb.kv')
    uk_emb = KeyedVectors.load('hw3/temp_data/uk_emb.kv')

In [7]:
# https://radimrehurek.com/gensim/models/keyedvectors.html
if False:
    ru_emb.save('hw3/temp_data/ru_emb.kv')
    uk_emb.save('hw3/temp_data/uk_emb.kv')

Посмотрим на пару серпень-август (являющихся переводом)

In [8]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [9]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [10]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

In [11]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("hw3/content/ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("hw3/content/ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)
mapping.score(X_train, Y_train)

0.5359725792059653

In [13]:
y_hat = mapping.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(Y_test, y_hat), mean_squared_error(Y_test, y_hat)

(0.03386641, 0.0025563438)

Проверим, куда перейдет `серпень`:

In [14]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531433939933777),
 ('июнь', 0.8402522802352905),
 ('март', 0.8385884165763855),
 ('сентябрь', 0.8331485390663147),
 ('февраль', 0.8311209678649902),
 ('октябрь', 0.8278020024299622),
 ('ноябрь', 0.8243728876113892),
 ('июль', 0.8229618072509766),
 ('август', 0.8112280368804932),
 ('январь', 0.8022986650466919)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [15]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    #assert len(pairs) == len(ru_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in ru_emb.most_similar([mapped_vector], topn=topn)]:
            num_matches += 1 
        
    precision_val = num_matches / len(pairs)
    return precision_val

In [16]:
ru_emb.most_similar(august.reshape(1, -1), topn=5)

[('апрель', 0.8531433939933777),
 ('июнь', 0.8402522802352905),
 ('март', 0.8385884165763855),
 ('сентябрь', 0.8331485390663147),
 ('февраль', 0.8311209678649902)]

In [17]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [18]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [30]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

AssertionError: 

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [21]:
def learn_transform(X_train, Y_train):
    from numpy import linalg as LA
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, s, V = LA.svd(np.matmul(X_train.T, Y_train))
    W_ = np.matmul(U,V)
    
    return W_

In [22]:
W = learn_transform(X_train, Y_train)

In [23]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245132565498352),
 ('июнь', 0.8056628704071045),
 ('сентябрь', 0.8055762052536011),
 ('март', 0.8032935857772827),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946796417236328),
 ('ноябрь', 0.7939637303352356),
 ('август', 0.7938188314437866),
 ('февраль', 0.7923860549926758),
 ('декабрь', 0.7715375423431396)]

In [24]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

AssertionError: 

### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [25]:
with open("hw3/content/fairy_tale.txt", "r",encoding='UTF-8') as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [26]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """    
    result = ""
    words = sentence.split(" ")
    translated_words = []
    for word in words:
        try:
            emb = uk_emb[word]
            translated_words.append(ru_emb.most_similar([np.matmul(emb, W)], topn=1)[0][0])
        except:
            translated_words.append(word)
    result = " ".join(translated_words) 
    return result

In [27]:
translate("кіт зловив мишу")

'кот поймал мышку'

In [28]:
assert translate(".") == ". "
assert translate("1 , 3") == "1 , 3 "
assert translate("кіт зловив мишу") == "кот поймал мышку "

AssertionError: 

In [29]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к обеих бабы добывать огнь ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи самогончик со маком , сладкий , че и прогнула , чтобы он успокоился ; а лисичка ой и підгледала , че токмо что бабка свесилась во печь , чтобы прость огня , то лисичка сейчас ухватила самогончик че и дунула со хаты , че , пробежать , весь мак со его кринки , 

# Дополнительные материалы

## Почитать
### База:  
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[Deep Learning, NLP, and Representations, Christopher Olah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)  

### Как кластеризовать смыслы многозначных слов:  
[Making Sense of Word Embeddings (2016), Pelevina et al](http://anthology.aclweb.org/W16-1620)    

### Как оценивать эмбеддинги
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel](http://www.aclweb.org/anthology/D15-1036)  
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu](https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)  
[Problems With Evaluation of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui](https://arxiv.org/pdf/1605.02276.pdf)  
[Improving Reliability of Word Similarity Evaluation by Redesigning Annotation Task and Performance Measure (2016), Oded Avraham, Yoav Goldberg](https://arxiv.org/pdf/1611.03641.pdf)  
[Evaluating Word Embeddings Using a Representative Suite of Practical Tasks (2016), N. Nayak](https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)  


## Посмотреть
[Word Vector Representations: word2vec, Lecture 2, cs224n](https://www.youtube.com/watch?v=ERibwqs9p38)